In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/microtext/Dataset - tachygraphy.csv


**Text Preprocessing**

In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv("/kaggle/input/microtext/Dataset - tachygraphy.csv")
df.shape

(4958, 3)

In [4]:
df.head()

,Text,Meaning,Sentiment
0,Last session of the day http://twitpic.com/67ezh,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely --...,Shanghai is also really exciting (precisely --...,positive
2,submit the report ASAP!,submit the report as soon as possilbe,negative
3,happy bday!,Happy Birthday!,positive
4,The OGs - I like it!!,The original gangsters - i like it!,positive


In [5]:
df=df.drop('Meaning',axis=1)
df = df.rename(columns={'Text': 'tweet'})
df = df.rename(columns={'Sentiment': 'class'})
df['class'] = df['class'].str.lower()
sentiment_counts = df['class'].value_counts()
sentiment_counts

class
neutral     1743
positive    1643
negative    1572
Name: count, dtype: int64

In [6]:
df.isnull().sum()

tweet    1
class    0
dtype: int64

In [7]:
df=df.fillna('')

In [8]:
import re
from bs4 import BeautifulSoup

def text_cleaning(text):
    text = str(text)
    soup = BeautifulSoup(text, "html.parser")    ###removing html tages
    text = re.sub(r'\[[^]]*\]', '', soup.get_text())      ##removing text within square brackets
    pattern = r"[^a-zA-Z0-9\s,']"                  # Removing unwanted characters
    text = re.sub(pattern, '', text)
    url_pattern = r'http\S+|www\S+'            ###removing urls
    text = re.sub(url_pattern, '', text)
    return text

In [9]:
df['tweet']=df['tweet'].apply(text_cleaning).tolist()

/tmp/ipykernel_34/2827669698.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")    ###removing html tages


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

2024-08-18 05:17:48.814135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 05:17:48.814241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 05:17:48.934599: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [13]:
df['tweet'].dtype

dtype('O')

In [14]:
df['class'].dtype

dtype('O')

In [15]:
df.tail(10)

,tweet,class
4948,time flies by when you're facetiming the bae,positive
4949,U dont know how I feel about this,negative
4950,come to my job I'll give you a baconator lol,positive
4951,Rihanna just made Maya Angelou her avi you're...,negative
4952,im the lil bitch in the theaters tryna hold ba...,negative
4953,make a pet face wtf wrong with me tonight haha,negative
4954,I dnt care anymore Boyz ain't worth d drama,negative
4955,No relationship is perfect tho me bae goo from...,negative
4956,Over here tryna get my nail polishes and shit lol,negative
4957,No one was loved d way i luv U,positive


In [16]:
df = df[df['class'] != "neutral"]  ###droping neutral columns
df['tweet'] = df['tweet'].apply(lambda x: x.lower())

print(df[ df['class'] == 'positive'].size)
print(df[ df['class'] == 'negative'].size)

for idx,row in df.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['tweet'].values)
X = tokenizer.texts_to_sequences(df['tweet'].values)
X = pad_sequences(X)

3286
3144


/tmp/ipykernel_34/1180210649.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt',' ')
/tmp/ipykernel_34/1180210649.py:8: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt',' ')


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

max_features = 5000  

embed_dim = 128 
lstm_out = 64  

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))  # Assuming 2 classes for classification

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [18]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [19]:
Y = pd.get_dummies(df['class']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2572, 29) (2572, 2)
(643, 29) (643, 2)


In [20]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
81/81 - 8s - 100ms/step - accuracy: 0.6128 - loss: 0.6585
Epoch 2/10
81/81 - 3s - 35ms/step - accuracy: 0.7834 - loss: 0.4457
Epoch 3/10
81/81 - 3s - 35ms/step - accuracy: 0.8639 - loss: 0.3130
Epoch 4/10
81/81 - 6s - 71ms/step - accuracy: 0.9001 - loss: 0.2362
Epoch 5/10
81/81 - 3s - 35ms/step - accuracy: 0.9351 - loss: 0.1719
Epoch 6/10
81/81 - 3s - 34ms/step - accuracy: 0.9495 - loss: 0.1445
Epoch 7/10
81/81 - 3s - 36ms/step - accuracy: 0.9603 - loss: 0.1068
Epoch 8/10
81/81 - 3s - 35ms/step - accuracy: 0.9638 - loss: 0.0903
Epoch 9/10
81/81 - 3s - 35ms/step - accuracy: 0.9786 - loss: 0.0690
Epoch 10/10
81/81 - 3s - 36ms/step - accuracy: 0.9759 - loss: 0.0807


In [21]:
validation_size = 500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test,verbose = 2, batch_size = batch_size)
print("loss_score: %.2f" % (score))
print("accuracy: %.2f" % (acc))

5/5 - 0s - 70ms/step - accuracy: 0.7413 - loss: 1.0362
loss_score: 1.04
accuracy: 0.74


In [22]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 206ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 28ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 2

In [23]:
twt = ['I been told you that yo folks dont care about you . Im starving and my dad deadass didnt make me a plate']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    1   79  477    7   18  559 1000
    39  275   54    7   10 1914    6    5  458  165  118   13    4 1563]]
1/1 - 0s - 199ms/step
negative


In [24]:
model.save('lstm_model.h5')